<a href="https://colab.research.google.com/github/anamacias9850-arch/Proyecto-Integrador-3er.-semestre/blob/main/Proyecto_Matematicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análisis de los factores determinantes de la probabilidad de inclusión laboral formal en adultos mayores mediante Regresión Logística Binaria**

**INTEGRANTES DEL EQUIPO**

Ana Elena Macías Amezcua

Edgar Raymundo González Torres

Patricia Jaime Pérez


**TEMA:**
Empleabilidad en las personas adultas mayores: Factores determinantes para la inclusión en el mercado laboral formal vs. ocupación voluntaria en Jalisco.


**OBJETIVO PRINCIPAL:**
Desarrollar un modelo probabilístico que permita identificar qué características del perfil del solicitante (Edad, Escolaridad, Salud, Género) aumentan matemáticamente la probabilidad de acceder a un Empleo Formal (con sueldo fijo) frente a la alternativa de ser Empacador Voluntario.

**OBTENCIÓN DE DATOS:**

**Fuente:** Elaboración propia. Base de datos Vinculación Productiva Jalisco (2024-2025).

**Estrategia de extracción:** Consulta directa

**Variables a observar:**

Variable Objetivo ($Y$): Tipo de Colocación (Binaria).
$1 =$ Empleo Formal (Auxiliar, Chofer, Ventas, etc.).
$0 =$ Empleo Informal (Empacador Voluntario).

Variables Predictoras ($X$): \
Edad.\
Nivel de Estudios (¿Más estudios garantiza trabajo formal?).\
Salud / Discapacidad.\
Género.\
Tiene Pensión (¿El que tiene pensión prefiere ser empacador por flexibilidad?).

Volumen de datos: 657



Estructura final:

Elegimos esta base y el objetivo porque:

Impacto Social: Estás analizando la calidad del empleo, no solo el dinero.
Técnica: Usar Regresión Logística es un paso muy elegante en Ciencia de Datos.
Resultados: Podremos decir conclusiones como: "Por cada 5 años más de edad, la probabilidad de conseguir empleo formal baja un 15%, empujándolos a ser empacadores"

La base de datos se guardará en formato .csv

## Importación de datos



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#import pandas as pd
#df = pd.read_csv("/content/BASE DATOS EN PROYECTO VP 2024-2025 _AEMA.CSV.csv")


In [3]:
import pandas as pd
df = pd.read_excel("BASE DATOS EN PROYECTO  VP 2024-2025  JALISCO_AEMA.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'BASE DATOS EN PROYECTO  VP 2024-2025  JALISCO_AEMA.xlsx'

In [ ]:
print(df.columns)

In [ ]:
df.head()

## Limpieza inicial:

LIMPIEZA Y PREPARACIÓN DE DATOS
Manejo de valores faltantes:
Se identificará el porcentaje de datos faltantes por variable.

Transformación de la Variable Objetivo (Y):
Se analizará la columna de TIPO DE EMPLEO AL QUE SE VINCULÓ. Se aplicará una función lógica donde todos los registros que contengan la cadena "INFORMAL" o vacio se codifican como 0, y el resto ("FORMAL") se codifican como 1.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df['TIPO DE EMPLEO AL QUE SE VINCULÓ'].unique()

In [ ]:
#Transformacion de variable dependiente
#df['EMPLEO_FORMAL'] = df['TIPO DE EMPLEO AL QUE SE VINCULÓ'].map({'FORMAL': 1, 'INFORMAL': 0})
df['EMPLEO_FORMAL'] = (df['TIPO DE EMPLEO AL QUE SE VINCULÓ'] == 'FORMAL').astype(int)
df = df.drop(columns=['TIPO DE EMPLEO AL QUE SE VINCULÓ'])

In [ ]:
#Visualizacion de cuantas personas obtuvieron un empleo formal
df['EMPLEO_FORMAL'].value_counts()

## Transformación de Variables Predictoras:
Estudios: Se convertirá a escala numérica jerárquica para medir la correlación entre preparación académica y tipo de empleo.
Edad: Se mantendrá numérica para evaluar si existe una "barrera de edad" donde, a partir de cierto año, la probabilidad de empleo formal cae drásticamente.

Transformaciones de variables:
Se crearán variables dummy para características categóricas (ej: "Sí/No" para género).  
Las variables cuantitativas no requieren escalarse porque todas están dadas en la misma escala (0-100).

In [ ]:
#Busqueda de valores únicos
df['ESTUDIOS'].unique()

In [ ]:
mapeo_estudios = {
    'OTRO': 0,
    'PRIMARIA INCOMPLETA': 0,
    'SABE LEER Y ESCRIBIR': 0,
    'NO SABE LEER Y ESCRIBIR': 0,
    'PRIMARIA COMPLETA': 1,
    'SECUNDARIA INCOMPLETA': 1,
    'SECUNDARIA COMPLETA': 2,
    'PREPARATORIA INCOMPLETA': 2,
    'CARRERA COMERCIAL': 2,
    'CARRERA TÉCNICA': 2,
    'PREPARATORIA COMPLETA': 3,
    'LICENCIATURA INCOMPLETA': 3,
    'LICENCIATURA COMPLETA': 4,
    'MAESTRÍA': 5


}

# Aplicar el mapeo a una nueva columna
df['ESTUDIOS_ORDINAL'] = df['ESTUDIOS'].map(mapeo_estudios)

In [ ]:

df['SEXO_FEMENINO'] = df['SEXO'].map({'FEMENINO': 1, 'MASCULINO': 0})

# Mapeo simple para PENSIONADO (columna PENSIONADO) y COMPUTACION
df['TIENE_PENSION'] = df['PENSIONADO'].map({'SI': 1, 'NO': 0})
df['SABE_COMPUTACION'] = df['COMPUTACIÓN'].map({'SI': 1, 'NO': 0, 'NO ': 0})

# Creación de dummy para Discapacidad
# Si TIPO DE DISCAPACIDAD es 'NINGUNA', entonces 0, de lo contrario 1.
df['TIENE_DISCAPACIDAD'] = (df['TIPO DE DISCAPACIDAD'] != 'NINGUNA').astype(int)

# Eliminar las columnas originales para evitar redundancia
df = df.drop(columns=['SEXO', 'PENSIONADO', 'COMPUTACIÓN', 'TIPO DE DISCAPACIDAD', 'ESTUDIOS'])

In [ ]:
df['EMPLEO_FORMAL'].unique()

In [ ]:
nuevas_columnas = ['SEXO_FEMENINO', 'TIENE_PENSION', 'SABE_COMPUTACION', 'TIENE_DISCAPACIDAD', 'ESTUDIOS_ORDINAL']
for columna in nuevas_columnas:
    print(df[columna])

## Variables a Eliminar (Ruido):
Se eliminarán variables administrativas (Fechas, Folios) y datos de contacto (Teléfono, Domicilio). También se evaluará eliminar CUANTO DESEA GANAR si presenta alta colinealidad con el tipo de puesto (es subjetivo lo que cada persona desea ganar, parece obvio que un empacador espera ganar menos que un gerente), para centrar el modelo en las características socio_demográficas de la persona.

In [ ]:
#Eliminacion de variables que no se consideraran para el modelo

df = df.drop(columns=['MUNICIPIO', 'AÑO', 'C.P.', 'NACIO EN ', 'ÁREA DE CARRERA O ESTUDIOS', 'EXPERIENCIA', 'PUESTO', 'PUESTO SOLICITADO', 'GUSTA VENTAS', 'REGIMEN DE PENSIÓN ', 'MONTO DE LA PENSIÓN', 'CUANTO GANÓ ANTES', 'SERVICIO MÉDICO', 'REQUIERE COTIZAR', '65  MÁS', 'IDIOMAS', 'VINCULADO A '])
df = df.drop(columns=['FECHA DE NAC.', 'PROB. SALUD', 'TOMA MEDICAMENTOS ', 'CUANTO DESEA GANAR ', 'OBSERVACIONES', 'HABILIDADES', 'ULTIMO EMPLEO', 'NÚMERO DE MEDICAMENTOS QUE TOMA'])

# La siguiente celda es para guardar el excel y ver que tal va la manipulación del mismo

In [ ]:
#Guardar un nuevo excel con las variables agregadas
nuevo_nombre_archivo = "NUEVAS VARIABLES BASE DATOS EN PROYECTO  VP 2024-2025  JALISCO_AEMA.xlsx"

try:
    df.to_excel(nuevo_nombre_archivo, index=False)
    print(f"\n✅ ¡Éxito! El nuevo dataset se ha guardado en el archivo: '{nuevo_nombre_archivo}'")
except Exception as e:
    print(f"\n❌ Error al guardar el archivo Excel: {e}")

## Regresión Lineal Múltiple

Una vez que se limpiaron los datos al máximo, estamos revisando de nuevo para validar la congruencia matemática y el modelo de Machine Learning

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
formula = "EMPLEO_FORMAL ~ EDAD + ESTUDIOS_ORDINAL + SEXO_FEMENINO + TIENE_PENSION + SABE_COMPUTACION + TIENE_DISCAPACIDAD"
modelo = smf.ols(formula, data=df)
resultados = modelo.fit()

In [ ]:
print("\n--- Resumen del Modelo de Regresión Lineal Múltiple ---")
print(resultados.summary())
print("-" * 91)

In [ ]:
print("Coeficientes del Modelo y su Interpretación (p < 0.05)")
coeficientes = resultados.params
p_values = resultados.pvalues

In [ ]:
# Iterar sobre los coeficientes y p-valores para imprimir solo los significativos
for var, coef in coeficientes.items():
    p_val = p_values[var]
    if p_val < 0.05:
        # Formatear el coeficiente a 4 decimales
        coef_redondeado = round(coef, 4)

        if var == 'Intercept':
            print(f"**Intercepto:** {coef_redondeado}")
            print(f"  > Este es el valor esperado de EMPLEO_FORMAL cuando todas las variables predictoras son cero.")
        else:
            print(f"**{var}:** Coeficiente = {coef_redondeado} (P-valor < 0.05)")
            if coef_redondeado > 0:
                print(f"  > Un aumento de una unidad en **{var}** se asocia con un aumento de {abs(coef_redondeado)} en la probabilidad de tener EMPLEO_FORMAL, manteniendo las otras variables constantes.")
            else:
                print(f"  > Un aumento de una unidad en **{var}** se asocia con una disminución de {abs(coef_redondeado)} en la probabilidad de tener EMPLEO_FORMAL, manteniendo las otras variables constantes.")
    else:
        # Variables no significativas al 5%
        print(f"**{var}:** No es estadísticamente significativo (P-valor > 0.05).")

Consideramos que:
El título y el objetivo están alineados con la lógica matemática rigurosa:"Regresión Logística Binaria":
Correcto, la variable respuesta ($y$) es dicotómica (Formal vs. Voluntario/Informal).
Probabilidad: porque queremos una predicción absoluta (Sí/No), sino un puntaje de propensión ($0 \le p \le 1$).
Y, Factores Determinantes: Nuestra interpretación en matemáticas es óptima para encontrar los coeficientes $\beta$ (pesos) que tienen mayor magnitud en la ecuación.

Paso 1: Fundamentación Visual (La Curva Sigmoide)
Nuestro objetivo es ver cómo características (como la edad) empujan a la persona hacia el empleo formal.
A diferencia de una línea recta, usaremos esto:

Shutterstock

Eje X (Horizontal): La combinación del perfil (Edad + Estudios + Salud...).

Eje Y (Vertical): La Probabilidad de Empleo Formal (de 0 a 1).

La Curva: Notamos cómo el cambio no es lineal. Quizás tener primaria o secundaria no cambia mucho, pero brincar a Preparatoria/Universidad dispara la probabilidad (la parte empinada de la curva).


Este gráfico es fundamental para nuestro marco teórico porque visualiza exactamente cómo el modelo matemático transforma los datos (Edad, Estudios, etc.) en una probabilidad entre 0 y 1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Definir la función matemática Sigmoide (La fórmula de la Regresión Logística)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 2. Generar datos teóricos para el Eje X
# Imaginemos que este eje representa el "Puntaje del Perfil" (suma de edad, estudios, etc.)
x = np.linspace(-10, 10, 100)
y = sigmoid(x)

# 3. Crear el Gráfico
plt.figure(figsize=(10, 6))
plt.plot(x, y, color='blue', linewidth=3, label='Función Logística (Sigmoide)')

# 4. Añadir líneas de referencia y decoraciones pedagógicas
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Umbral de Decisión (0.5)')
plt.axvline(x=0, color='gray', linestyle=':', alpha=0.5)

# Etiquetas personalizadas para tu proyecto
plt.title('Fundamento Matemático: La Curva de Probabilidad de Empleo', fontsize=14)
plt.ylabel('Probabilidad de Empleo Formal (0 a 1)', fontsize=12)
plt.xlabel('← Menor Perfil (Voluntario)       Puntaje Combinado (z)       Mayor Perfil (Formal) →', fontsize=10)

# Anotaciones explicativas en el gráfico
plt.text(-8, 0.1, 'Zona de Baja Probabilidad\n(Empacador Voluntario)', fontsize=10, color='green')
plt.text(4, 0.9, 'Zona de Alta Probabilidad\n(Empleo Formal)', fontsize=10, color='navy')

plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

Paso 2: Validación de Hipótesis con nuestros Datos
Vamos a ejecutar el código para confirmar matemáticamente qué variables son realmente "significativas" en la base de datos 3.0.

Usamos las variables que mencionamos en el objetivo:

Edad (EDAD)

Escolaridad (ESTUDIOS_ORDINAL)

Salud (TIENE_DISCAPACIDAD)

Género (SEXO_FEMENINO)

Plus: Agregaremos SABE_COMPUTACION, ya que suele ser un discriminante fuerte en el mercado formal.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Cargar datos
df = pd.read_csv('3.0 BASE DATOS EN PROYECTO _AEMA.csv')

# 2. SUB-OBJETIVO: Identificar correlaciones (Fuerza de relación lineal)
# Seleccionamos las columnas de interés basadas en tu objetivo
cols_interes = ['EMPLEO_FORMAL', 'EDAD', 'ESTUDIOS_ORDINAL',
                'TIENE_DISCAPACIDAD', 'SEXO_FEMENINO', 'SABE_COMPUTACION']

plt.figure(figsize=(10, 8))
# Heatmap: El color rojo/azul intenso indica un factor determinante fuerte
sns.heatmap(df[cols_interes].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Mapa de Calor: ¿Qué variables determinan el Empleo Formal?', fontsize=14)
plt.show()

# 3. SUB-OBJETIVO: Visualizar el impacto de la EDAD (Variable Continua)
plt.figure(figsize=(10, 6))
sns.boxplot(x='EMPLEO_FORMAL', y='EDAD', data=df, palette='Set2')
plt.title('Distribución de EDAD según Tipo de Empleo (0=Voluntario, 1=Formal)')
plt.xlabel('Empleo Formal (0 = No, 1 = Sí)')
plt.ylabel('Edad (Años)')
plt.grid(True, alpha=0.3)
plt.show()

# 4. Interpretación Estadística Rápida
correlacion_estudios = df['EMPLEO_FORMAL'].corr(df['ESTUDIOS_ORDINAL'])
print(f"Coeficiente de correlación Empleo-Estudios: {correlacion_estudios:.4f}")

In [ ]:
# Calcular solo el dato numérico exacto
dato = df['EMPLEO_FORMAL'].corr(df['ESTUDIOS_ORDINAL'])
print(f"El valor de correlación es: {dato}")

0.346 es una "Luz Verde" matemática.

En Ciencias Sociales y Humanidades (donde analizamos comportamiento humano y no leyes físicas), una correlación superior a 0.30 se considera estadísticamente relevante. Este es el caso de la obtención o no de una actividad productiva en personas mayores.

¿Qué significa esto para nosotros?
 Que hemos demostrado cuantitativamente que el nivel de estudios sí empuja la probabilidad hacia el empleo formal.
 No es una garantía absoluta (si fuera 1.0, todos los universitarios tendrían empleo formal automáticamente), pero es un factor de peso ("driver").

Fase 3:
** Entrenamiento del Modelo** (Encontrar la Ecuación)
Ahora que validamos las variables, vamos a construir la Regresión Logística.
No solo le diremos a la computadora "aprende", sino que le pediremos que nos devuelva la ecuación matemática que descubrió.La ecuación se verá así:$$P(y=1) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 \cdot \text{Edad} + \beta_2 \cdot \text{Estudios} + \dots)}}$$Los valores $\beta$ (betas) son los que vamos va a calcular ahora mismo.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# 1. Definir nuestras variables X (Predictoras) e y (Objetivo)
features = ['EDAD', 'ESTUDIOS_ORDINAL', 'SEXO_FEMENINO',
            'TIENE_PENSION', 'SABE_COMPUTACION', 'TIENE_DISCAPACIDAD']

X = df[features]
y = df['EMPLEO_FORMAL']

# 2. División de Datos: 80% Entrenamiento (Aprender), 20% Prueba (Evaluar)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Estandarización (Paso CLAVE en Álgebra Lineal)
# Ajustamos las escalas para que la 'Edad' no domine sobre las variables de 0 y 1.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Entrenamiento del Modelo (Descenso del Gradiente)
modelo = LogisticRegression()
modelo.fit(X_train_scaled, y_train)

# 5. REVELACIÓN MATEMÁTICA: Los Coeficientes (Betas)
print("--- LA ECUACIÓN DEL MODELO ---")
coeficientes = pd.DataFrame(modelo.coef_.T, index=features, columns=['Peso (Beta)'])
print(coeficientes.sort_values(by='Peso (Beta)', ascending=False))

print("\n--- EVALUACIÓN (¿Qué tan bien predice?) ---")
y_pred = modelo.predict(X_test_scaled)
print(f"Exactitud (Accuracy): {modelo.score(X_test_scaled, y_test):.2%}")

Valores Positivos (+): Aumentan la probabilidad de empleo formal. (Espero ver ESTUDIOS_ORDINAL y SABE_COMPUTACION aquí arriba).

Valores Negativos (-): Disminuyen la probabilidad (te mandan al voluntariado). ¿Dónde quedó la EDAD?

Tener una Exactitud (Accuracy) del 83.97% en un estudio social es altísimo.
Significa que el modelo matemático entiende correctamente la situación laboral de 84 de cada 100 adultos mayores basándose solo en esas 6 preguntas.

1. Los "Motores" de la Inclusión (Coeficientes Positivos)
Estas son las variables que empujan la probabilidad hacia el 1 (Empleo Formal).

ESTUDIOS_ORDINAL (+0.463): La Variable Reina del Modelo.

Matemáticas: Es el coeficiente más grande. Tiene el mayor peso vectorial en la decisión.

Interpretación: La educación formal es el discriminante #1. Cada grado académico extra aumenta drásticamente la probabilidad de formalidad.

SABE_COMPUTACION (+0.426):

Matemáticas: Casi tan fuerte como los estudios.

Interpretación: Esto es un hallazgo clave para 2024-2025. La brecha digital es una barrera real. Saber usar una computadora es casi un pasaporte automático a la formalidad, independientemente de la edad.

TIENE_PENSION (+0.057):

Matemáticas: Un valor muy cercano a cero.

Interpretación: Irrelevante. Estadísticamente, tener o no pensión no decide si consiguen empleo formal. Es una variable "ruido".

2. Las "Barreras" Estructurales (Coeficientes Negativos)
Estas variables restan puntos a la ecuación, empujando a la persona hacia el 0 (Empacador Voluntario/Informal).

SEXO_FEMENINO (-0.326): 🚩 Hallazgo Crítico.

Matemáticas: Un coeficiente negativo fuerte.

Interpretación: Existe una brecha de género. A igualdad de edad y estudios, ser mujer reduce matemáticamente la probabilidad de acceder a un empleo formal en comparación con un hombre. Esto es una conclusión social potente.

EDAD (-0.268):

Interpretación: Confirma el edadismo. Por cada año que pasa (desviación estándar), el mercado laboral formal cierra un poco más la puerta.

TIENE_DISCAPACIDAD (-0.259):

Interpretación: Otra barrera estructural significativa, casi del mismo peso que la edad.

Para el Modelo Final
La ecuación predictiva final ($Z$) que pondremos en nuestro reporte es esta:$$Z = \beta_0 + 0.46(\text{Estudios}) + 0.42(\text{Comp}) - 0.27(\text{Edad}) - 0.32(\text{Mujer}) - 0.26(\text{Discap})$$Y la probabilidad de empleo formal ($P$) se calcula pasando esa $Z$ por la función sigmoide:$$P(\text{Formal}) = \frac{1}{1 + e^{-(Z)}}$$(Nota: $\beta_0$ es el "intercepto", un número base que el modelo calcula internamente, pero los coeficientes que mostramos son los que explican el "cambio").


In [ ]:
import matplotlib.pyplot as plt

# Configuración del lienzo
plt.figure(figsize=(10, 4))
plt.axis('off')  # Ocultar ejes, solo queremos la fórmula

# La Fórmula en lenguaje matemático (LaTeX)
formula_matematica = r"$P(Y=1|X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 X_1 + ... + \beta_n X_n)}}$"

# Dibujar la fórmula en el centro
plt.text(0.5, 0.5, formula_matematica, fontsize=28, ha='center', va='center')

plt.title("Fórmula de la Probabilidad (Regresión Logística)", fontsize=14, color='gray')
plt.show()

**Un ejemplo:**
Simulador:

Podemos inventar dos perfiles ficticios:

Don Roberto: 70 años, Primaria, No sabe computación.

Doña Elena: 70 años, Preparatoria, Sí sabe computación.

In [ ]:
# --- SIMULADOR DE INCLUSIÓN LABORAL ---
import pandas as pd
import numpy as np

# 1. Definimos los perfiles ficticios
# Orden de variables: [EDAD, ESTUDIOS, MUJER?, PENSIÓN?, COMPUTACIÓN?, DISCAPACIDAD?]

# Referencia de Estudios: 1=Primaria, 2=Secundaria, 3=Preparatoria, 4=Universidad
# Referencia Binaria: 0=No, 1=Sí

# CASO A: Don Roberto (70 años, Primaria, Hombre, No Computación)
roberto = [70, 1, 0, 1, 0, 0]

# CASO B: Doña Elena (70 años, Prepa, Mujer, Sí Computación)
elena   = [70, 3, 1, 1, 1, 0]

# 2. Convertimos a DataFrame (La estructura que entiende el modelo)
columnas = ['EDAD', 'ESTUDIOS_ORDINAL', 'SEXO_FEMENINO',
            'TIENE_PENSION', 'SABE_COMPUTACION', 'TIENE_DISCAPACIDAD']

perfiles = pd.DataFrame([roberto, elena], columns=columnas, index=['Don Roberto', 'Doña Elena'])

# 3. IMPORTANTE: Estandarización (Álgebra Lineal)
# Debemos aplicar la misma transformación matemática que usamos al entrenar.
# Usamos .transform() y NO .fit_transform() porque usamos la media/desviación del entrenamiento.
perfiles_scaled = scaler.transform(perfiles)

# 4. Predicción de Probabilidades (La Magia)
# .predict_proba nos da dos números: [Probabilidad de NO, Probabilidad de SÍ]
probs = modelo.predict_proba(perfiles_scaled)

# 5. Mostrar Resultados
print("--- RESULTADOS DEL SIMULADOR ---")
for i, nombre in enumerate(perfiles.index):
    prob_exito = probs[i][1] * 100 # Tomamos la columna 1 (Probabilidad de 'Sí')
    print(f"{nombre}: {prob_exito:.2f}% de probabilidad de Empleo Formal")

# 6. Visualización Rápida
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 4))
plt.bar(perfiles.index, probs[:, 1], color=['gray', 'blue'])
plt.ylabel('Probabilidad (0 a 1)')
plt.title('Comparativa de Perfiles: Impacto de Educación y Tecnología')
plt.ylim(0, 1) # Escala de 0 a 100%
plt.axhline(y=0.5, color='red', linestyle='--', label='Umbral de Decisión')
plt.legend()
plt.show()

Si miramos la brecha entre ambos porcentajes.

Probablemente Don Roberto tenga una probabilidad baja (¿cercana al 10-30%?), lo que lo condena matemáticamente al voluntariado.

Doña Elena, a pesar de tener la misma edad y la "desventaja" estadística que vimos por género, debería dispararse hacia arriba gracias al peso positivo (+0.46) de los Estudios y (+0.42) de la Computación.

Miremos los números con "lupa matemática":$$\text{Brecha} = 42.88\% - 21.28\% = \mathbf{21.6 \text{ puntos porcentuales}}$$Interpretación para tu Nuestra teoría:
Tenemos los 3 argumentos clave que estos números validan:
El Poder del Capital Humano (x2):Doña Elena tiene casi el doble (2x) de probabilidad que Don Roberto de encontrar trabajo formal.
Conclusión: La combinación de Educación Media + Alfabetización Digital es capaz de duplicar las oportunidades de un adulto mayor, mitigando otras barreras.La "Penalización" de la Edad (El Techo de Cristal).

Ojo a esto: A pesar de que Elena está muy preparada, no llega al 50%. Se quedó en 42.88%.¿Por qué? Porque el coeficiente negativo de la EDAD (70 años) pesa muchísimo ($\beta \approx -0.27$).

Realidad: El modelo nos dice que, a los 70 años, el mercado laboral es tan duro que incluso alguien preparado como Elena tiene difícil cruzar la barrera hacia la formalidad.

La Brecha de Género vs. Capacitación:Elena tenía una desventaja matemática por ser mujer ($\beta \approx -0.32$), pero sus habilidades (Computación $\beta \approx +0.42$) fueron suficientes para anular esa desventaja y superar a Roberto.

La pregunta: ¿El modelo se equivoca? (La Matriz de Confusión)
Ya tenemos la ecuación y las predicciones.
Pero para que nuestro trabajo sea científicamente robusto, debemos responder: "De los que el modelo predijo que eran formales, ¿cuántos lo eran realmente?"

En medicina y ciencias sociales, esto es vital. No queremos decirle a alguien que tiene oportunidad (Falso Positivo) si no la tiene.

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 1. Generar la matriz comparando Realidad (y_test) vs. Predicción (y_pred)
matriz = confusion_matrix(y_test, y_pred)

# 2. Visualizarla
plt.figure(figsize=(6, 5))
sns.heatmap(matriz, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicho: Voluntario', 'Predicho: Formal'],
            yticklabels=['Real: Voluntario', 'Real: Formal'])

plt.title('Matriz de Confusión: ¿Dónde se equivoca el modelo?', fontsize=14)
plt.ylabel('Verdad (Datos Reales)')
plt.xlabel('Predicción del Modelo')
plt.show()

# 3. Reporte de Métricas Finales
print(classification_report(y_test, y_pred))

Estos números nos dicen la verdad desnuda sobre el modelo.
Aunque la exactitud general era alta (84%), este reporte te muestra dónde es más importante.

**"Discusión de Resultados":**

1. El Desbalance de la Realidad (Support)
Observemos en la columna de la derecha (Support):

Clase 0 (Voluntarios): 100 personas.

Clase 1 (Formales): 31 personas.

**Conclusión: **

La base de datos refleja una realidad social dura.
Hay 3 veces más adultos mayores en trabajos informales/voluntarios que en formales.
Al tener menos ejemplos de "éxito" (Clase 1), al modelo le cuesta más aprender a detectarlos.

2. Análisis de la Clase 1 (Empleo Formal)
Aquí es donde debes poner el foco, porque es tu objetivo principal.

Precision (0.68):

¿Qué significa? De todas las veces que el modelo dijo "Esta persona consigue empleo formal", acertó el 68% de las veces.

**Lectura: Es un modelo cauteloso. No promete empleo formal a la ligera.**

Recall (0.61):

¿Qué significa? De todas las personas adultas mayores que realmente tenían empleo formal, el modelo solo detectó al 61%.

El "Punto Ciego":

Se le escaparon casi 4 de cada 10 (el 39%). El modelo los clasificó erróneamente como voluntarios porque sus perfiles (quizás por edad avanzada) se parecían mucho a la mayoría.

Tu Matriz de Confusión ReconstruidaBasado en esos porcentajes, tu matriz visual (los cuadros azules) se ve aproximadamente así:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# --- PASO 1: CARGAR DATOS (CON LA RUTA CORRECTA) ---
# Usamos la ruta exacta que me diste
archivo = '/content/3.0 BASE DATOS EN PROYECTO _AEMA.csv'

try:
    df = pd.read_csv(archivo)
    print("✅ Archivo cargado correctamente.")
except FileNotFoundError:
    print("❌ Aún no encuentra el archivo. Asegúrate de subirlo al menú de la carpeta a la izquierda.")

# Definir variables
features = ['EDAD', 'ESTUDIOS_ORDINAL', 'SEXO_FEMENINO',
            'TIENE_PENSION', 'SABE_COMPUTACION', 'TIENE_DISCAPACIDAD']
X = df[features]
y = df['EMPLEO_FORMAL']

# --- PASO 2: ENTRENAMIENTO (Álgebra Lineal y Cálculo) ---
# Dividir 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar (Normalizar la matriz)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo logístico
modelo = LogisticRegression()
modelo.fit(X_train_scaled, y_train)
y_pred = modelo.predict(X_test_scaled)

# --- PASO 3: VISUALIZACIÓN PARA EL CARTEL ---
plt.figure(figsize=(12, 5))

# A. Gráfico de la Matriz de Confusión
plt.subplot(1, 2, 1)
matriz = confusion_matrix(y_test, y_pred)
sns.heatmap(matriz, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicho: Voluntario', 'Predicho: Formal'],
            yticklabels=['Real: Voluntario', 'Real: Formal'])
plt.title('Matriz de Confusión', fontsize=14)
plt.ylabel('Realidad')
plt.xlabel('Predicción del Modelo')

# B. Tabla de Métricas (Visual)
# Hacemos un truco para mostrar la tabla como gráfico
plt.subplot(1, 2, 2)
plt.axis('off') # Apagar ejes
reporte = classification_report(y_test, y_pred, output_dict=True)
tabla = pd.DataFrame(reporte).transpose().round(2)
tabla_texto = plt.table(cellText=tabla.values, colLabels=tabla.columns, rowLabels=tabla.index,
                        loc='center', cellLoc='center', colWidths=[0.2]*4)
tabla_texto.auto_set_font_size(False)
tabla_texto.set_fontsize(10)
plt.title('Métricas de Desempeño', fontsize=14)

plt.tight_layout()
plt.show()

# --- PASO 4: DATOS TEXTUALES ---
acc = reporte['accuracy']
f1 = reporte['1']['f1-score']

print(f"\n🌟 DATOS FINALES PARA COPIAR AL CARTEL:")
print(f"-> Exactitud Global (Accuracy): {acc:.2%}")
print(f"-> F1-Score (Capacidad de detectar Empleo Formal): {f1:.2f}")